In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

# Set up Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # run in headless mode
chrome_options.add_argument("--no-sandbox")  # bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
chrome_options.add_argument("--disable-gpu")  # applicable to windows os only
chrome_options.add_argument("start-maximized")  # open Browser in maximized mode
chrome_options.add_argument("disable-infobars")  # disabling infobars
chrome_options.add_argument("--disable-extensions")  # disabling extensions


def scraperLoop(key, text_list, driver, numberOfObjects, special_word = '', time_of_waiting = 4):
    
    time.sleep(1)
    chat_xpath = '/html/body/div/div[1]/div/div[2]/div/form/div/div/textarea'
    chat_input = driver.find_element(By.XPATH, chat_xpath)
    chat_input.send_keys(f'No yapping {numberOfObjects} synonyms and similar words of {key} in Mandarin? {special_word}. Separate words in spaces.')
    chat_input.send_keys(Keys.RETURN)
    time.sleep(time_of_waiting)
    for i in range(20):
        try: 
            fetch_xpath = '/html/body/div/div[1]/div/div[1]/div/div[2]/div[1]'
            fetch_output = driver.find_element(By.XPATH, fetch_xpath)
            output = re.sub(r"[（.*?）0-9a-zA-Z]", '', fetch_output.text)
            text_list[key] = re.split(r"[.\s,，。]+", output)
            break
        except Exception as e:
            time.sleep(1)
    new_chat_xpath = '/html/body/div/div[1]/nav[3]/div[1]/a[2]'
    new_chat_entry = driver.find_element(By.XPATH, new_chat_xpath)
    new_chat_entry.click()

In [11]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
time.sleep(1)
driver.get("https://twllm.com/")

b1_xpath = '/html/body/div[2]/div/div/div/div/button'
b1_input = driver.find_element(By.XPATH, b1_xpath)
b1_input.click()

text_key = ['白色', '紅色', '大(容量)','中(形容容量)','小', '舉例各種顏色']
text_list = {}
for i in range(len(text_key)):
    key = text_key[i]
    scraperLoop(key, text_list, driver, 10)
driver.quit()


In [10]:
text_list


{'白色': ['白色', '白皙', '純白', '雪白', '亮白', '淺色', '淡色', '粉飾', '無瑕疵', '無暇'],
 '紅色': ['紅色', '紅的', '赤色', '丹紅', '朱红', '大红', '洋红', '玫瑰红', '深红', '粉色'],
 '大(容量)': ['巨大', '容量大', '大容量', '大型', '容積大', '大規模', '可觀', '巨型', '高容量', '超大'],
 '中': ['在', '中文里', '"中"', '的同義詞和相似詞有：中心、中央、其中、當中、中間、中部、核心、中點、中途', ''],
 '小': ['小的', '小巧', '微小', '纖細', '迷你', '袖珍', '小型', '小巧玲瓏', '嬌小', '可愛'],
 '舉例各種顏色': ['舉例各種顏色：紅色、橙色、黃色、綠色、藍色、紫色、黑色、白色']}